In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

2025-02-14 09:37:24.761085: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-14 09:37:24.774189: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739515044.788958  697461 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739515044.793392  697461 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-14 09:37:24.809414: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Настройки
model_name = "Qwen/Qwen2.5-14B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка модели и токенизатора
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,  # Включение оптимизации
    device_map=device,
    torch_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,padding_side='left')

# Создаем пайплайн БЕЗ явного указания устройства
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    #max_new_tokens = 500,
    max_length=2000,
    temperature=1.0,
    #do_sample=False,
    pad_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cuda


In [13]:
def getPerson(n):
    answer=[]
    prompt = [{"role":"user", "content": "Ты эксперт в генерации личностей-клиентов для симуляции тестирования системы по автоматизации колл-центра в компании, которая предоставляет услуги интернет связи. Сгенерируй 1.Пол персоны. 2. Фамилию Имя Отчество. 3. Возраст персоны. 4. Паспортные данные. 5. Проблему(Компания занимаеться услугами предоставления интернета). 6.Открытую информацию(То есть ту информацию ,которые нам сказал клиент при звонке). 7. Закрытую информацию (Информацию ,которую только можно узнать во время разговора с оператором). Сгенерирй только необходимые данные."}]
    for _ in range(n):
        answer.append(pipe(prompt, return_full_text=False)[0]['generated_text'])
    return answer

In [14]:
getPerson(1)

['Я понимаю ваши требования и готов предложить примерную персону для тестирования системы автоматизации колл-центра. Однако стоит отметить, что при реальном тестировании или работе важно учитывать вопросы конфиденциальности и безопасности данных.\n\n### Персона:\n1. **Пол:** Мужской\n2. **Фамилия Имя Отчество:** Иванов Игорь Сергеевич\n3. **Возраст:** 35 лет\n4. **Паспортные данные:** № 9999 999999, выдан 01.01.2010 года УМВД России по г. Москве\n\n### Открытая информация (то, что клиент может сказать при звонке):\n5. **Проблема:** Интернет отключился несколько дней назад, а теперь работает медленно.\n6. **Открытая информация:** "Здравствуйте, меня зовут Игорь. У меня возникла проблема со скоростью подключения к интернету, которое стало заметно снижаться. Также было отключение сервиса на несколько дней."\n\n### Закрытая информация (то, что можно узнать во время разговора с оператором):\n7. **Закрытая информация:** Оператор может выяснить, что у Игоря Сергеевича идет проблема с маршрути